# FINN - End-to-End Flow
-----------------------------------------------------------------


In [50]:
import netron
from finn.util.visualization import showSrc, showInNetron
from finn.util.basic import make_build_dir
import os
    
build_dir = "Fast-QNN/outputs/txaviour/AlexNet"

In [51]:
import torch
import onnx
from finn.util.test import get_test_model_trained
from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup

alexnet = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
export_onnx_path = build_dir+"/tfc_w1_a1.onnx"
export_qonnx(alexnet, torch.rand(1, 1, 32, 32), build_dir+"/tfc_w1_a1.onnx")
qonnx_cleanup(export_onnx_path, out_file=export_onnx_path)

Using cache found in /tmp/home_dir/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [75]:
showInNetron(build_dir+"/tfc_w1_a1.onnx", port=5000)

Serving 'Fast-QNN/outputs/txaviour/AlexNet/tfc_w1_a1.onnx' at http://0.0.0.0:5000


In [55]:
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
model = ModelWrapper(build_dir+"/tfc_w1_a1.onnx")
model = model.transform(ConvertQONNXtoFINN())

/tmp/home_dir/.local/lib/python3.10/site-packages/qonnx/transformation/gemm_to_matmul.py:57: UserWarning: The GemmToMatMul transformation only offers explicit support for version 9 of the Gemm node, but the ONNX version of the supplied model is 14. Thus the transformation may fail or return incomplete results.
  warnings.warn(


In [56]:
model.save(build_dir+"/tfc_w1_a1_finn.onnx")
showInNetron(build_dir+"/tfc_w1_a1_finn.onnx")

OSError: [Errno 98] Address already in use

In [57]:
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.fold_constants import FoldConstants

model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

model.save(build_dir+"/tfc_w1_a1_tidy.onnx")

In [26]:
showInNetron(build_dir+"/tfc_w1_a1_tidy.onnx")

Stopping http://0.0.0.0:8081
Serving 'Fast-QNN/outputs/initial/tfc_w1_a1_tidy.onnx' at http://0.0.0.0:8081


In [58]:
from finn.util.pytorch import ToTensor
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.core.datatype import DataType

model = ModelWrapper(build_dir+"/tfc_w1_a1_tidy.onnx")
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
# preprocessing: torchvision's ToTensor divides uint8 inputs by 255
totensor_pyt = ToTensor()
chkpt_preproc_name = build_dir+"/tfc_w1_a1_preproc.onnx"
export_qonnx(totensor_pyt, torch.randn(ishape), chkpt_preproc_name)
qonnx_cleanup(chkpt_preproc_name, out_file=chkpt_preproc_name)
pre_model = ModelWrapper(chkpt_preproc_name)
pre_model = pre_model.transform(ConvertQONNXtoFINN())

# join preprocessing and core model
model = model.transform(MergeONNXModels(pre_model))
# add input quantization annotation: UINT8 for all BNN-PYNQ models
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])

model.save(build_dir+"/tfc_w1_a1_with_preproc.onnx")
showInNetron(build_dir+"/tfc_w1_a1_with_preproc.onnx")

/tmp/home_dir/.local/lib/python3.10/site-packages/qonnx/transformation/infer_data_layouts.py:127: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


OSError: [Errno 98] Address already in use

In [59]:
from qonnx.transformation.insert_topk import InsertTopK

# postprocessing: insert Top-1 node at the end
model = model.transform(InsertTopK(k=1))
chkpt_name = build_dir+"/tfc_w1_a1_pre_post.onnx"
# tidy-up again
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())
model.save(chkpt_name)

showInNetron(build_dir+"/tfc_w1_a1_pre_post.onnx")

OSError: [Errno 98] Address already in use

In [60]:
from finn.transformation.streamline import Streamline
showSrc(Streamline)

class Streamline(Transformation):
    """Apply the streamlining transform, see arXiv:1709.04060."""

    def apply(self, model):
        streamline_transformations = [
            ConvertSubToAdd(),
            ConvertDivToMul(),
            BatchNormToAffine(),
            ConvertSignToThres(),
            MoveMulPastMaxPool(),
            MoveScalarLinearPastInvariants(),
            AbsorbSignBiasIntoMultiThreshold(),
            MoveAddPastMul(),
            MoveScalarAddPastMatMul(),
            MoveAddPastConv(),
            MoveScalarMulPastMatMul(),
            MoveScalarMulPastConv(),
            MoveAddPastMul(),
            CollapseRepeatedAdd(),
            CollapseRepeatedMul(),
            MoveMulPastMaxPool(),
            AbsorbAddIntoMultiThreshold(),
            FactorOutMulSignMagnitude(),
            AbsorbMulIntoMultiThreshold(),
            Absorb1BitMulIntoMatMul(),
            Absorb1BitMulIntoConv(),
            RoundAndClipThresholds(),
        ]
        for tr

In [61]:
from finn.transformation.streamline.reorder import MoveScalarLinearPastInvariants
import finn.transformation.streamline.absorb as absorb

model = ModelWrapper(build_dir+"/tfc_w1_a1_pre_post.onnx")
# move initial Mul (from preproc) past the Reshape
model = model.transform(MoveScalarLinearPastInvariants())
# streamline
model = model.transform(Streamline())
model.save(build_dir+"/tfc_w1_a1_streamlined.onnx")
showInNetron(build_dir+"/tfc_w1_a1_streamlined.onnx")

OSError: [Errno 98] Address already in use

In [63]:
from qonnx.transformation.bipolar_to_xnor import ConvertBipolarMatMulToXnorPopcount
from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.general import RemoveUnusedTensors

model = model.transform(ConvertBipolarMatMulToXnorPopcount())
model = model.transform(absorb.AbsorbAddIntoMultiThreshold())
model = model.transform(absorb.AbsorbMulIntoMultiThreshold())
# absorb final add-mul nodes into TopK
model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
model = model.transform(RoundAndClipThresholds())

# bit of tidy-up
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())

model.save(build_dir+"/tfc_w1a1_ready_for_hw_conversion.onnx")
showInNetron(build_dir+"/tfc_w1a1_ready_for_hw_conversion.onnx")

OSError: [Errno 98] Address already in use

In [64]:
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
model = ModelWrapper(build_dir+"/tfc_w1a1_ready_for_hw_conversion.onnx")
model = model.transform(to_hw.InferBinaryMatrixVectorActivation())
# TopK to LabelSelect
model = model.transform(to_hw.InferLabelSelectLayer())
# input quantization (if any) to standalone thresholding
model = model.transform(to_hw.InferThresholdingLayer())
model.save(build_dir+"/tfc_w1_a1_hw_layers.onnx")
showInNetron(build_dir+"/tfc_w1_a1_hw_layers.onnx")

OSError: [Errno 98] Address already in use

In [73]:
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition, PrepareIP

model = ModelWrapper(build_dir+"/tfc_w1_a1_hw_layers.onnx")
parent_model = model.transform(CreateDataflowPartition())
parent_model = parent_model.transform(PrepareIP())
parent_model.save(build_dir+"/tfc_w1_a1_dataflow_parent.onnx")
showInNetron(build_dir+"/tfc_w1_a1_dataflow_parent.onnx")

NameError: name 'netron' is not defined

In [66]:
from qonnx.custom_op.registry import getCustomOp
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
dataflow_model_filename = sdp_node.get_nodeattr("model")
showInNetron(dataflow_model_filename)

IndexError: list index out of range

In [39]:
model = ModelWrapper(dataflow_model_filename)

In [40]:
thresh_node = model.get_nodes_by_op_type("Thresholding")[0]
thresh_node_inst = getCustomOp(thresh_node)
thresh_node_inst.set_nodeattr("preferred_impl_style", "hls")

In [41]:
# print the names of the supported PYNQ boards
from finn.util.basic import pynq_part_map
print(pynq_part_map.keys())

dict_keys(['Ultra96', 'Ultra96-V2', 'Pynq-Z1', 'Pynq-Z2', 'ZCU102', 'ZCU104', 'ZCU111', 'RFSoC2x2', 'RFSoC4x2', 'KV260_SOM'])


In [42]:
# change this if you have a different PYNQ board, see list above
pynq_board = "Pynq-Z2"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10

In [43]:
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers
model = model.transform(SpecializeLayers(fpga_part))

model.save(build_dir+"/tfc_w1_a1_specialize_layers.onnx")
showInNetron(build_dir+"/tfc_w1_a1_specialize_layers.onnx")

Stopping http://0.0.0.0:8081
Serving 'Fast-QNN/outputs/initial/tfc_w1_a1_specialize_layers.onnx' at http://0.0.0.0:8081


In [44]:
fc0 = model.graph.node[1]
fc0w = getCustomOp(fc0)

print("CustomOp wrapper is of class " + fc0w.__class__.__name__)

fc0w.get_nodeattr_types()

CustomOp wrapper is of class MVAU_hls


{'PE': ('i', True, 0),
 'SIMD': ('i', True, 0),
 'MW': ('i', True, 0),
 'MH': ('i', True, 0),
 'resType': ('s', False, 'auto', {'auto', 'dsp', 'lut'}),
 'ActVal': ('i', False, 0),
 'inputDataType': ('s', True, ''),
 'weightDataType': ('s', True, ''),
 'outputDataType': ('s', True, ''),
 'accDataType': ('s', False, 'INT32'),
 'binaryXnorMode': ('i', False, 0, {0, 1}),
 'noActivation': ('i', False, 0, {0, 1}),
 'numInputVectors': ('ints', False, [1]),
 'mem_mode': ('s',
  False,
  'internal_decoupled',
  {'external', 'internal_decoupled', 'internal_embedded'}),
 'ram_style': ('s', False, 'auto', {'auto', 'block', 'distributed', 'ultra'}),
 'ram_style_thresholds': ('s',
  False,
  'auto',
  {'auto', 'block', 'distributed'}),
 'runtime_writeable_weights': ('i', False, 0, {0, 1}),
 'backend': ('s', True, 'fpgadataflow'),
 'preferred_impl_style': ('s', False, '', {'', 'hls', 'rtl'}),
 'code_gen_dir_ipgen': ('s', False, ''),
 'ipgen_path': ('s', False, ''),
 'ip_path': ('s', False, ''),
 'ip_

In [46]:
fc_layers = model.get_nodes_by_op_type("MVAU_hls")
# (PE, SIMD, in_fifo_depth, out_fifo_depth, ramstyle) for each layer
config = [
    (16, 49, [16], [64], "block"),
    (8, 8, [64], [64], "auto"),
    (8, 8, [64], [64], "auto"),
    (10, 8, [64], [10], "distributed"),
]
for fcl, (pe, simd, ififo, ofifo, ramstyle) in zip(fc_layers, config):
    fcl_inst = getCustomOp(fcl)
    fcl_inst.set_nodeattr("PE", pe)
    fcl_inst.set_nodeattr("SIMD", simd)
    fcl_inst.set_nodeattr("inFIFODepths", ififo)
    fcl_inst.set_nodeattr("outFIFODepths", ofifo)
    fcl_inst.set_nodeattr("ram_style", ramstyle)
    
# set parallelism for input quantizer to be same as first layer's SIMD
inp_qnt_node = model.get_nodes_by_op_type("Thresholding_hls")[0]
inp_qnt = getCustomOp(inp_qnt_node)
inp_qnt.set_nodeattr("PE", 49)

In [47]:
model.save(build_dir+"/tfc_w1_a1_set_folding_factors.onnx")
showInNetron(build_dir+"/tfc_w1_a1_set_folding_factors.onnx")

Stopping http://0.0.0.0:8081
Serving 'Fast-QNN/outputs/initial/tfc_w1_a1_set_folding_factors.onnx' at http://0.0.0.0:8081


In [49]:
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
model = ModelWrapper(build_dir+"/tfc_w1_a1_set_folding_factors.onnx")
model = model.transform(ZynqBuild(platform = pynq_board, period_ns = target_clk_ns))

/home/fastqnn/finn/src/finn/transformation/fpgadataflow/floorplan.py:107: UserWarning: 10 nodes have no entry in the provided floorplan, SLR was set to -1
  warnings.warn(


KeyboardInterrupt: 

In [48]:
from finn.transformation.fpgadataflow.make_pynq_driver import MakePYNQDriver
model = model.transform(MakePYNQDriver("zynq-iodma"))

AssertionError: 
                Ensure CreateDataflowPartition called before driver creation.

In [ ]:
model.save(build_dir + "/tfc_w1_a1_post_synthesis.onnx")

In [ ]:
showInNetron(build_dir + "/tfc_w1_a1_post_synthesis.onnx")

In [ ]:
model = ModelWrapper(build_dir + "/tfc_w1_a1_post_synthesis.onnx")
sdp_node_middle = getCustomOp(model.graph.node[1])
postsynth_layers = sdp_node_middle.get_nodeattr("model")

showInNetron(postsynth_layers)

In [ ]:
model = ModelWrapper(postsynth_layers)
model.model.metadata_props

In [ ]:
model = ModelWrapper(build_dir + "/tfc_w1_a1_post_synthesis.onnx")
model.model.metadata_props

In [17]:
! ls {model.get_metadata_prop("vivado_pynq_proj")}

advanced  basics  end2end_example  Fast-QNN  LFCW1A1.onnx


In [ ]:
from shutil import copy
from distutils.dir_util import copy_tree

# create directory for deployment files
deployment_dir = make_build_dir(prefix="pynq_deployment_")
model.set_metadata_prop("pynq_deployment_dir", deployment_dir)

# get and copy necessary files
# .bit and .hwh file
bitfile = model.get_metadata_prop("bitfile")
hwh_file = model.get_metadata_prop("hw_handoff")
deploy_files = [bitfile, hwh_file]

for dfile in deploy_files:
    if dfile is not None:
        copy(dfile, deployment_dir)

# driver.py and python libraries
pynq_driver_dir = model.get_metadata_prop("pynq_driver_dir")
copy_tree(pynq_driver_dir, deployment_dir)

In [ ]:
from pkgutil import get_data
import onnx.numpy_helper as nph
import matplotlib.pyplot as plt

raw_i = get_data("qonnx.data", "onnx/mnist-conv/test_data_set_0/input_0.pb")
x = nph.to_array(onnx.load_tensor_from_string(raw_i))
plt.imshow(x.reshape(28,28), cmap='gray')

In [ ]:
import numpy as np

model = ModelWrapper(build_dir + "/tfc_w1_a1_post_synthesis.onnx")
iname = model.graph.input[0].name
oname = parent_model.graph.output[0].name
ishape = model.get_tensor_shape(iname)
print("Expected network input shape is " + str(ishape))
np.save(deployment_dir + "/input.npy", x.reshape(ishape))

In [ ]:
! ls {deployment_dir}

In [ ]:
from shutil import make_archive
make_archive('deploy-on-pynq-tfc', 'zip', deployment_dir)

```shell
unzip deploy-on-pynq-tfc.zip -d finn-tfc-demo
cd finn-tfc-demo
sudo python3 -m pip install bitstring
sudo python3 driver.py --exec_mode=execute --batchsize=1 --bitfile=resizer.bit --inputfile=input.npy
```